# Using Jurassic-2 Grande Instruct on SageMaker through Model Packages

This sample notebook shows you how to deploy **Jurassic-2 Grande Instruct** using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. Before running this notebook, please make sure you got this notebook from the model catalog on SageMaker AWS Management Console.
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**.
1. This notebook is intended to work with **boto3 v1.25.4** or higher.

## Contents:
1. [Select model package](#1.-Select-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Interact with the model](#B.-Interact-with-the-model)
   3. [Give it an instruction](#C.-Give-it-an-instruction)
   4. [Create your own product description!](#D.-Create-your-own-product-description!)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Select model package
Confirm that you received this notebook from the model catalog in SageMaker AWS Management Console.

In [1]:
model_package_map = {
    "us-east-1": "arn:aws:sagemaker:us-east-1:416407187090:model-package/j2-grande-instruct-v1-0-20"
}

In [2]:
import json
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import sagemaker as sage
import boto3

### Check the version of boto3 - must be v1.25.4 or higher
If you see a lower version number, pick another kernel to run the notebook, with Python 3.8 or above

In [3]:
boto3.__version__

'1.26.74'

### Install ai21 python SDK

In [4]:
! pip install -U "ai21[SM]"
import ai21

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Preparing metadata (setup.py) ... done
  Created wheel for ai21: filename=ai21-1.0.2-py3-none-any.whl size=15548 sha256=dc1f4ff22a3b5713faddac7068b8014356b2a5bf3d263cb11e8c1039bdba7e95
  Stored in directory: /home/ec2-user/.cache/pip/wheels/7d/c9/1b/f6fa60c4ec9d74273fefb9ba4aca6ccc70ac3fdf740e597cd8
Successfully built ai21


In [5]:
region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise ("UNSUPPORTED REGION")

model_package_arn = model_package_map[region]

In [6]:
role = get_execution_role()
sagemaker_session = sage.Session()

runtime_sm_client = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

In [7]:
model_name = "j2-grande-instruct"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.g5.12xlarge"
)

### A. Create an endpoint

In [9]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name, 
                         model_data_download_timeout=3600,
                         container_startup_health_check_timeout=600,
                        )

---------------!

Once endpoint has been created, you would be able to perform real-time inference.

### B. Interact with the model

You can think of Jurassic-2 Grande Instruct as a smart auto-completion algorithm: give it some text as input and it will generate some relevant text to naturally complete your input.

These two helpful concepts are worth being familiar with:
- **Prompt** - the input you provide to the model.
- **Completion** - the output text the model generates.

Enter a simple prompt: "To be, or", and let the model complete it

In [11]:
response = ai21.Completion.execute(sm_endpoint="j2-grande-instruct",
                                   prompt="To be, or",
                                   maxTokens=4,
                                   temperature=0,
                                   numResults=1)

print(response['completions'][0]['data']['text'])

 not to be: that is the question


As you can see, the model identifies the beginning of a famous quote, and completes it correctly.

### C. Give it an instruction

This model was specifically trained to follow natural language instructions. It is the most natural way to interact with large language models: simply tell the model what you want it to do, and it will follow.

For this notebook, we will apply the model to creating a product description for an eCommerce site. Simply tell the model what you want it to do with a simple instruction.

In [13]:
instruction = """Write an engaging product description for clothing eCommerce site.
Product: Humor Men's Graphic T-Shirt.
Description:

"""

response = ai21.Completion.execute(sm_endpoint="j2-grande-instruct",
                                   prompt=instruction,
                                   maxTokens=100,
                                   temperature=0,
                                   numResults=1)

print(response['completions'][0]['data']['text'])

This funny t-shirt is perfect for anyone who enjoys a good laugh. It's made of 100% cotton and is soft and comfortable. The graphic on the front is bright and colorful, and it's sure to get attention. Whether you're wearing it to the gym, out with friends, or just around the house, this t-shirt is sure to make you smile.


#### Adjust the parameters
A useful parameter is the temperature. **You can increase creativity by tweaking the temperature.** With temperature 0, the model will always choose the most probable completion, so it will always be the same. Increasing the temperature will provide varying completions, where the completion may be different with every generation. We recommend starting with a value of 0.7.

In [14]:
instruction = """Write an engaging product description for clothing eCommerce site.
Product: Humor Men's Graphic T-Shirt.
Description:

"""

response = ai21.Completion.execute(sm_endpoint="j2-grande-instruct",
                                   prompt=instruction,
                                   maxTokens=100,
                                   temperature=0.7,
                                   numResults=3)

# Note that multiple completions - this is due to numResults > 1
for comp in response['completions']:
    print(comp['data']['text'].strip())
    print("=============")

This funny t-shirt is perfect for anyone who enjoys a good joke. It's made from a soft cotton blend and is comfortable to wear. The graphic on the front features a humorous saying, and it's sure to make people laugh. This shirt is perfect for casual wear or special occasions.
This t-shirt is perfect for anyone who wants to show off their sense of humor. The design features a funny saying on the front that will make you laugh. It's made of high-quality cotton, and it's comfortable to wear.

Whether you're going out with friends or just relaxing at home, this t-shirt is a must-have. Order yours today!
When life gives you lemons, make lemonade. But when life gives you lemons and lemonade, make lemon meringue pie. And that's exactly what this shirt is designed to do. This funny men's graphic t-shirt is perfect for anyone who enjoys a good joke and a delicious dessert. The shirt features an illustration of a lemon meringue pie, and the text "When life gives you lemons and lemonade, make lem

#### Be specific in your prompt
You can see that the model generated several versions. There may be times when you are looking for a stricter format - such as a single paragraph. Just ask the model to stick to it:

In [16]:
instruction = """Write an engaging product description for clothing eCommerce site. Make it a single paragraph.
Product: Humor Men's Graphic T-Shirt.
Description:

"""

response = ai21.Completion.execute(sm_endpoint="j2-grande-instruct",
                                   prompt=instruction,
                                   maxTokens=100,
                                   temperature=0.5,
                                   numResults=3)

for comp in response['completions']:
    print(comp['data']['text'].strip())
    print("=============")

This funny t-shirt is perfect for anyone who likes to make jokes. It's made of 100% cotton and it's available in sizes S to 3XL. The t-shirt features a graphic of an old man with a cane and the words "Old Fart" printed on the front. It's sure to make your friends laugh!
This funny men's graphic t-shirt is perfect for anyone who enjoys a good laugh. The shirt features an image of a dog wearing a cone on its head, and the caption reads, "I can't touch my toes, but I can lick my own balls." It's sure to make people laugh and is perfect for any occasion.
This funny t-shirt is perfect for the dad who loves to joke around. The shirt features a graphic of a bottle of beer and a lime with the words "Dad Jokes" printed below. It's sure to get a few laughs at the next family gathering.


#### Personalize your prompt
The model can generate nice descriptions based on only an instruction, but they may not be aligned with your actual product. If you have special features, ask the model to include them:

In [18]:
instruction = """Write an engaging product description for clothing eCommerce site. Make sure to include the following features in the description.
Product: Humor Men's Graphic T-Shirt with a print of Einstein's quote: "artificial intelligence is no match for natural stupidity”
Features:
- Soft cotton
- Short sleeve
Description:

"""

response = ai21.Completion.execute(sm_endpoint="j2-grande-instruct",
                                   prompt=instruction,
                                   maxTokens=100,
                                   temperature=0.7,
                                   numResults=3)

for comp in response['completions']:
    print(comp['data']['text'].strip())
    print("=============")

This funny graphic t-shirt is perfect for anyone who wants to show their sense of humor. It's made of soft cotton and it's comfortable to wear all day long.
This funny Einstein t-shirt is perfect for anyone who enjoys a good joke. The print is bright and colorful, and it's sure to get people talking. The shirt is made from soft cotton, and it's comfortable to wear.

If you're looking for a unique and stylish t-shirt, the Humor Men's Graphic T-Shirt is the perfect choice.
This t-shirt is perfect for anyone who wants to look stylish while showing off their sense of humor. The print is high quality and won't fade after washing. The shirt is made from soft cotton and fits true to size. It's perfect for wearing on a lazy day or for going out with friends.


#### More format restrictions?
If you have several examples of your optimal output, you can feed the model with a few-shot prompt that includes several examples. Prompts like this are very useful when you want to enforce a certain format that is non-trivial to pick up from just one example. These prompts work best with base models; try to use them with [Jurassic-2 Grande](https://github.com/AI21Labs/SageMaker/blob/main/J2_Grande_example_model_use.ipynb) instead.

### D. Create your own product description!

Want to build your own product description? Here's your chance!

In [19]:
# Functions to build the prompt
def list_features(features):
    return "\n".join(
        f"- {feat}" for feat in features
    )

def build_prompt(product, features):
    example = "Write product descriptions for fashion eCommerce site. Make sure to include the following features in the description.\n"
    example += f"Product: {product}\n"
    example += "Features:\n"
    example += list_features(features)
    example += "\nDescription:\n"
    
    return example

In [20]:
product = input("Product: (for example, \'Women's Boho Beach Dress\')")
features = input("Features, separated by commas with no spaces: (for example, \'midi dress,swing hem,special for summer\')")
your_prompt = build_prompt(product, features.split(","))

Product: (for example, 'Women's Boho Beach Dress')Women's Boho Beach Dress
Features, separated by commas with no spaces: (for example, 'midi dress,swing hem,special for summer')midi dress,swing hem,special for summer


In [21]:
response = ai21.Completion.execute(sm_endpoint="j2-grande-instruct",
                                   prompt=your_prompt,
                                   maxTokens=100,
                                   temperature=0.7,
                                   numResults=3)

for comp in response['completions']:
    print(comp['data']['text'].strip())
    print("=============")

This boho beach dress is perfect for summer days! It's made of lightweight, flowy fabric and has a swing hem that moves with you. The midi length is perfect for warm weather, and the dress can easily be dressed up or down.
This midi dress is perfect for summer! It's made from lightweight cotton and comes in a variety of colors, so you can choose your favorite. The swing hem adds a touch of fun and style, and the dress is loose and comfortable.
This boho beach dress is the perfect choice for your next vacation! The midi length is flattering and swing hem adds a touch of movement. The fabric is perfect for summer, and you'll love the colorful print. Whether you're headed to the beach or just relaxing at home, this dress is a must-have.


### Interested in learning more?
Take a look at our [blog post](https://www.ai21.com/blog/building-cv-profile-generator-using-ai21-studio) to understand the process of building a good prompt.

### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [22]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Clean-up

### A. Delete the model

In [23]:
model.delete_model()